# Advanced Lane Finding Project Demo

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

This demo show how the pipeline works, with test images firstly and then with project video.

---
## Calibrate camera using chessboard images

In [2]:
import glob
import importlib

import matplotlib.pyplot as plt

import pipeline


debug = False

importlib.reload(pipeline)
chessboard_img_files = glob.glob("data/camera_cal/*.jpg")
chessboard_rgb_images = [pipeline.load_image(f) for f in chessboard_img_files]
undistorter = pipeline.Undistorter(chessboard_rgb_images, (9,6))

if debug:
    print("camera matrix:\n", undistorter.camera_matrix)
    print("distortion coefficients:\n", undistorter.distort_coeffs)
    img = pipeline.load_image("data/camera_cal/calibration1.jpg")
    undistorted = undistorter.apply(img)
    pipeline.save_image("output_images/undistorted_calibration1.jpg", undistorted)

## Undistort images using calibrated values

In [3]:
raw_image_files = glob.glob("data/test_images/*.jpg")
rgb_raw_images = [pipeline.load_image(f) for f in raw_image_files]
rgb_undistorted_images = [undistorter.apply(rgb) for rgb in rgb_raw_images]

if debug:
    plt.imshow(rgb_undistorted_images[0])
    pipeline.save_image("output_images/undistorted_test.jpg", rgb_undistorted_images[0])

## Apply threshold to images

In [4]:
# PARAMS
extractor = pipeline.LaneFeatureExtractor(
        ksize_dict={
            'abs_sobel': 5,
            'mag_sobel': 5,
            'dir_sobel': 15,
        },
        thresh_dict={
            'abs_sobel': (40,255),
            'mag_sobel': (40,255),
            'dir_sobel': (0.8,1.4),
            'satur': (120, 255),
        })

binary_images = [extractor.extract(rgb, debug=debug) for rgb in rgb_undistorted_images]

if debug:
    pipeline.save_image("output_images/binary_test.jpg", binary_images[0])

## Warp perspective into birdview

In [6]:
height = binary_images[0].shape[0]
warper = pipeline.Warper(
        upper_left_point_pair=[
            [526, 470], [300, 200]
        ],
        lower_left_point_pair=[
            [0, height-1], [300, height-1]
        ])

warped_binary_images = [warper.forward_warp(img, debug=debug) for img in binary_images]

if debug:
    pipeline.save_image("output_images/warped_binary_test.jpg", warped_binary_images[0])

## Detect lane and embed lane info on image

In [7]:
detector = pipeline.LaneDetector(
        n_windows=9,
        win_margin=60,
        reposition_thresh_rate=(0.02, 0.8),
        x_m_per_px=3.7/500,
        y_m_per_px=25/700,
        x_ignore_area=300,
        )

save_detected_test = True
result_images = []
for bin_img, raw_img in zip(warped_binary_images, rgb_undistorted_images):
    detect_image_file = None
    if debug and save_detected_test:
        detect_image_file = "output_images/detected_test.jpg"
    result_images.append(detector.draw_lane(bin_img, raw_img, warper, debug=debug, save_detection_to=detect_image_file))
    if save_detected_test:
        save_detected_test = False

if debug:
    pipeline.save_image("output_images/embeddted_test.jpg", result_images[0])

## Apply to the project video

In [19]:
# Construct pipeline for stream processing
# using undistorer, extractor, warper, detector
def process_stream_image(raw_image):
    undistorted = undistorter.apply(raw_image)
    binary = extractor.extract(undistorted)
    warped_binary = warper.forward_warp(binary)
    embedded = detector.draw_lane(warped_binary, undistorted, warper, stream=True)
    return embedded

In [20]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

input_video_file = "data/videos/project_video.mp4"
output_video_file = "output_videos/project_video.mp4"

In [ ]:
# Stream processing
in_clip = VideoFileClip(input_video_file)
out_clip = in_clip.fl_image(process_stream_image)
%time out_clip.write_videofile(output_video_file, audio=False)

t:   0%|          | 2/1260 [00:00<01:08, 18.47it/s, now=None]

Moviepy - Building video output_videos/project_video.mp4.
Moviepy - Writing video output_videos/project_video.mp4



t:  95%|█████████▍| 1193/1260 [02:00<00:06,  9.59it/s, now=None]

In [ ]:
HTML("""
<video width="{width}" height="{height}" controls>
  <source src="{file}">
</video>
""".format(file=output_video_file, width=480, height=270))

In [18]:
# DEBUG
head_clip = in_clip.subclip(0, 1)
out_head_clip = head_clip.fl_image(process_stream_image)
out_head_clip.write_videofile("debug.mp4", audio=False)

t:   3%|▎         | 2/75 [00:00<00:04, 18.21it/s, now=None]

Moviepy - Building video debug.mp4.
Moviepy - Writing video debug.mp4



Moviepy - Done !
Moviepy - video ready debug.mp4
